In [1]:
# -*- coding: utf-8 -*-

import random
from deap import base, creator, tools, algorithms
import numpy as np
import time

In [4]:
def genetic_search(eval_function, vector_size, population_size):
    """
    Performs genetic algoritm search using the given inputs
    """
    #TODO: allow to give a population for start (it will need to be mutated)
    start_time = time.time()
    # 利用 creator 物件, 建立問題類別
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)#設定評分標準
    #規則
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)#變數隨機1,0
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=vector_size)#產生Ｎ個變數
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)#多少個vector

    toolbox.register("evaluate", eval_function)#可否交配
    toolbox.register("mate", tools.cxTwoPoint)#交配方式
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.10)#每個變數發生突變的機會
    toolbox.register("select", tools.selTournament, tournsize=3)#隨機取3個，挑選最好的

    pop = toolbox.population(n=population_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, stats=stats, halloffame=hof, verbose=True)
    print("Best individual is: %s\nwith fitness: %s" % (hof[0], hof[0].fitness))
    print('The search took %i minutes.' % ((start_time-time.time())/60))
    return pop, logbook, hof

* population – A list of individuals.</br>
* toolbox – A Toolbox that contains the evolution operators.</br>
* cxpb – The probability of mating two individuals.</br>
* mutpb – The probability of mutating an individual.</br>
* ngen – The number of generation.</br>
* stats – A Statistics object that is updated inplace, optional.</br>
* halloffame – A HallOfFame object that will contain the best individuals, optional.</br>
* verbose – Whether or not to log the statistics.</br>

* 一. 挑選Ｎ(population_size)位父母</br>
* 二. 排序、父母兩兩配對</br>
* 三. 0.5的機率成功交配，成功留孩子、失敗留父母</br>
* 四. 0.5的機率成功配對後，判斷孩子是否比父母優秀，優秀留孩子、不優秀留父母</br>
* 五. Ｎ位新世代開始變異，0.2的機率會發生隨機變異</br>
* 六. 最後全部變數再發生一次變異，0.1的機率會0、1互換</br>
* 七. 新世代確定</br>

遺傳演算法的特點

　　1．遺傳演算法從問題解的中集開始嫂索，而不是從單個解開始。

　　這是遺傳演算法與傳統優化演算法的極大區別。傳統優化演算法是從單個初始值迭代求最優解的；容易誤入局部最優解。遺傳演算法從串集開始搜索，覆蓋面大，利於全局擇優。

　　2．遺傳演算法求解時使用特定問題的信息極少，容易形成通用演算法程式。

　　由於遺傳演算法使用適應值這一信息進行搜索，並不需要問題導數等與問題直接相關的信息。遺傳演算法只需適應值和串編碼等通用信息，故幾乎可處理任何問題。

　　3．遺傳演算法有極強的容錯能力

　　遺傳演算法的初始串集本身就帶有大量與最優解甚遠的信息；通過選擇、交叉、變異操作能迅速排除與最優解相差極大的串；這是一個強烈的濾波過程；並且是一個並行濾波機制。故而，遺傳演算法有很高的容錯能力。

　　4．遺傳演算法中的選擇、交叉和變異都是隨機操作，而不是確定的精確規則。

　　這說明遺傳演算法是採用隨機方法進行最優解搜索，選擇體現了向最優解迫近，交叉體現了最優解的產生，變異體現了全局最優解的覆蓋。

　　5．遺傳演算法具有隱含的並行性